# Notes
    You have to stop sabotaging yourself.
    Learn what you are good at and keep improving that.
    You are free to do whatever you want.
    Regardless what happens in your career.
    Stop accummulating pieces of papers.
    And start doing actual work.
    Look long and hard at kaggle competitions, 
    understand the rules, 
    and go at it.
    All you have to do is start.
    
    I now understand that models are functions
    each having their own approach.
    It does not necessarily mean one is better than the other.
    A model is as good as it gets depending on how you use it.
    For this dataset,
    RandomForest seems to provide the best score.
    But because I don't know how it actually works.
    I cannot, tweak it as much.
    I understand it works on an IF and THEN logic.
    But that is as far as I get.
    
    LinearRegression on the otherhand does not
    have the best score
    but it has the most potential.
    Because it is so basic.
    If I fully understand it
    and fully understand the dataset,
    I can tweak it to give me better results.
    
    
    








































In [65]:
import pandas as pd
import numpy as np
import seaborn as sns

In [66]:
df = pd.read_csv('input/train.csv')

In [67]:
num = [x for x in df.columns if df.dtypes[x] in ('int64', 'float')]
cat = [x for x in df.columns if df.dtypes[x] == 'object']
target = 'SalePrice'
num.remove(target)

In [68]:
from sklearn.model_selection import train_test_split
train, test = train_test_split(df, test_size=.2, random_state=0)

# Explore

In [69]:
corr = df.corr(numeric_only=True)
corr[target].sort_values(ascending=False)

SalePrice        1.000000
OverallQual      0.790982
GrLivArea        0.708624
GarageCars       0.640409
GarageArea       0.623431
TotalBsmtSF      0.613581
1stFlrSF         0.605852
FullBath         0.560664
TotRmsAbvGrd     0.533723
YearBuilt        0.522897
YearRemodAdd     0.507101
GarageYrBlt      0.486362
MasVnrArea       0.477493
Fireplaces       0.466929
BsmtFinSF1       0.386420
LotFrontage      0.351799
WoodDeckSF       0.324413
2ndFlrSF         0.319334
OpenPorchSF      0.315856
HalfBath         0.284108
LotArea          0.263843
BsmtFullBath     0.227122
BsmtUnfSF        0.214479
BedroomAbvGr     0.168213
ScreenPorch      0.111447
PoolArea         0.092404
MoSold           0.046432
3SsnPorch        0.044584
BsmtFinSF2      -0.011378
BsmtHalfBath    -0.016844
MiscVal         -0.021190
Id              -0.021917
LowQualFinSF    -0.025606
YrSold          -0.028923
OverallCond     -0.077856
MSSubClass      -0.084284
EnclosedPorch   -0.128578
KitchenAbvGr    -0.135907
Name: SalePr

In [70]:
train[cat].apply(lambda x:set(x))

MSZoning                                 {FV, RM, RH, C (all), RL}
Street                                                {Pave, Grvl}
Alley                                            {nan, Pave, Grvl}
LotShape                                      {IR1, IR3, IR2, Reg}
LandContour                                   {Lvl, Low, Bnk, HLS}
Utilities                                         {NoSeWa, AllPub}
LotConfig                      {FR3, FR2, Corner, CulDSac, Inside}
LandSlope                                          {Sev, Gtl, Mod}
Neighborhood     {Crawfor, StoneBr, Edwards, SWISU, Gilbert, No...
Condition1       {RRNn, Feedr, PosN, Artery, RRAn, RRAe, Norm, ...
Condition2                 {Feedr, PosN, Artery, RRAe, Norm, PosA}
BldgType                     {1Fam, Duplex, Twnhs, TwnhsE, 2fmCon}
HouseStyle       {2Story, 1Story, 2.5Fin, SLvl, 1.5Fin, SFoyer,...
RoofStyle               {Hip, Mansard, Gable, Gambrel, Shed, Flat}
RoofMatl         {Metal, Membran, WdShngl, WdShake, CompShg, T

In [71]:
train['ExterQual'].value_counts()

ExterQual
TA    729
Gd    388
Ex     39
Fa     12
Name: count, dtype: int64

# Pipeline

In [72]:
num_pk = [x for x in num if x not in ('Id')]
cat_pk = ['ExterQual', 'ExterCond', 'BsmtQual', 'BsmtCond'
          , 'BsmtExposure', 'BsmtFinType1', 'BsmtFinType2'
          , 'HeatingQC', 'KitchenQual', 'FireplaceQu', 'GarageQual', 'GarageCond', 'PoolQC']

In [73]:
blu = {
    'ExterQual': {'key': ['Ex','Gd','TA','Fa','Po'], 'val': [5,4,3,2,1]}
    , 'ExterCond': {'key': ['Ex','Gd','TA','Fa','Po'], 'val': [5,4,3,2,1]}
    , 'BsmtQual': {'key': ['Ex','Gd','TA','Fa','Po'], 'val': [5,4,3,2,1]}
    , 'BsmtCond': {'key': ['Ex','Gd','TA','Fa','Po'], 'val': [5,4,3,2,1]}
    , 'BsmtExposure': {'key': ['Gd','Av','Mn','No'], 'val': [3,2,1,0]}
    , 'BsmtFinType1': {'key': ['GLQ','ALQ','BLQ','Rec','LWQ','Unf'], 'val': [6,5,4,3,2,0]}
    , 'BsmtFinType2': {'key': ['GLQ','ALQ','BLQ','Rec','LWQ','Unf'], 'val': [6,5,4,3,2,0]}
    , 'HeatingQC': {'key': ['Ex','Gd','TA','Fa','Po'], 'val': [5,4,3,2,1]}
    , 'KitchenQual': {'key': ['Ex','Gd','TA','Fa','Po'], 'val': [5,4,3,2,1]}
    , 'FireplaceQu': {'key': ['Ex','Gd','TA','Fa','Po'], 'val': [5,4,3,2,1]}
    , 'GarageQual': {'key': ['Ex','Gd','TA','Fa','Po'], 'val': [5,4,3,2,1]}
    , 'GarageCond': {'key': ['Ex','Gd','TA','Fa','Po'], 'val': [5,4,3,2,1]}
    , 'PoolQC': {'key': ['Ex','Gd','TA','Fa','Po'], 'val': [5,4,3,2,1]}
}

In [74]:
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.preprocessing import OneHotEncoder, StandardScaler, PolynomialFeatures
from sklearn.impute import SimpleImputer
from sklearn.base import BaseEstimator, TransformerMixin

In [75]:
class DataSelect(BaseEstimator, TransformerMixin):
    def __init__(self, attributes):
        self.attributes = attributes
    def fit(self, X, y=None):
        return self
    def transform(self, X, y=None):
        return X[self.attributes]
    
class CustomEncode(BaseEstimator, TransformerMixin):
    def __init__(self, led):
        self.led = led
    def fit(self, X, y=None):
        return self
    def transform(self, X, y=None):
        dat = {}
        for itm in self.led:
            dat['y_'+itm] = X[itm].map({x:y for x,y in zip(self.led[itm]['key']
                                                               , self.led[itm]['val'])})
        return pd.DataFrame(dat).fillna(0)

In [76]:
num_pp = Pipeline([
    ('dataselect', DataSelect(num_pk))
    , ('impute', SimpleImputer(strategy='median'))
    , ('poly', PolynomialFeatures())
    , ('scaler', StandardScaler())
])

cat_pp = Pipeline([
    ('dataselect', DataSelect(cat_pk))
    , ('customencode', CustomEncode(blu))
    , ('poly', PolynomialFeatures())
    , ('scaler', StandardScaler())
])

pipe = FeatureUnion([
    ('numeric', num_pp)
    , ('categorical', cat_pp)
])

pipe.fit(train)
train_pre = pipe.transform(train)
test_pre = pipe.transform(test)

# Select Model

In [77]:
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.linear_model import Ridge, Lasso, ElasticNet


In [78]:
rid_reg = Ridge()
rid_reg.fit(train_pre, train[target])
print(np.sqrt(-cross_val_score(rid_reg, train_pre, train[target], scoring='neg_mean_squared_error').mean()))
print(np.sqrt(-cross_val_score(rid_reg, test_pre, test[target], scoring='neg_mean_squared_error').mean()))

54041.72153021133
56164.9518649737


In [79]:
# ela_reg = ElasticNet()
# ela_reg.fit(train_pre, train[target])
# print(np.sqrt(-cross_val_score(ela_reg, train_pre, train[target], scoring='neg_mean_squared_error').mean()))
# print(np.sqrt(-cross_val_score(ela_reg, test_pre, test[target], scoring='neg_mean_squared_error').mean()))

In [80]:
lin_reg = LinearRegression()
lin_reg.fit(train_pre, train[target])
print(np.sqrt(-cross_val_score(lin_reg, train_pre, train[target], scoring='neg_mean_squared_error').mean()))
print(np.sqrt(-cross_val_score(lin_reg, test_pre, test[target], scoring='neg_mean_squared_error').mean()))

9556750140869816.0
55238.00600139872


In [81]:
ran_reg = RandomForestRegressor(random_state=0)
ran_reg.fit(train_pre, train[target])
print(np.sqrt(-cross_val_score(ran_reg, train_pre, train[target], scoring='neg_mean_squared_error').mean()))
print(np.sqrt(-cross_val_score(ran_reg, test_pre, test[target], scoring='neg_mean_squared_error').mean()))

29164.188482546666


KeyboardInterrupt: 

In [ ]:
dec_reg = DecisionTreeRegressor(random_state=0)
dec_reg.fit(train_pre, train[target])
print(np.sqrt(-cross_val_score(dec_reg, train_pre, train[target], scoring='neg_mean_squared_error').mean()))
print(np.sqrt(-cross_val_score(dec_reg, test_pre, test[target], scoring='neg_mean_squared_error').mean()))

In [ ]:
model_base = lin_reg
model_base.get_params()

# Tune Model

In [ ]:
from sklearn.model_selection import GridSearchCV
params = {
    'random_state': [0]
    , 'n_estimators': [300, 600]
    , 'max_features': [1, 8, 12]
}

grid = GridSearchCV(model_base, param_grid=params, scoring='neg_mean_squared_error')
# grid.fit(train_pre, train[target])
# grid.best_params_

In [ ]:
# model_tune = RandomForestRegressor(random_state=0, n_estimators=300, max_features=12)
model_tune = Ridge()
model_tune.fit(train_pre, train[target]);

# Validation

In [ ]:
print('model_base'
, np.sqrt(-cross_val_score(model_base, train_pre, train[target], scoring='neg_mean_squared_error').mean())
, np.sqrt(-cross_val_score(model_base, test_pre, test[target], scoring='neg_mean_squared_error').mean())
)

In [ ]:
print('model_tune'
, np.sqrt(-cross_val_score(model_tune, train_pre, train[target], scoring='neg_mean_squared_error').mean())
, np.sqrt(-cross_val_score(model_tune, test_pre, test[target], scoring='neg_mean_squared_error').mean())
)

    ## Scoring
    # Encoded categorical that are ordered.
    # model_base 29581.92123767121 45637.69717668411
    # model_tune 28454.07259678302 44573.59896205597
    
    # Include Categorical.
    # model_base 29761.712624668104 45622.31938365163
    # model_tune 30398.282498063614 46175.65530935336
    
    # Baseline. numeric only.
    # model_base 30027.52817329832 44130.043234907316
    # model_tune 29193.731684252467 44415.39678678987
----------

# Submission

In [ ]:
samp = pd.read_csv('input/test.csv')
pd.DataFrame({
    'Id': samp['Id']
    , 'SalePrice': model_tune.predict(pipe.transform(samp))
}).to_csv('submission.csv', index=False)

# EDA 1
    # In linear regression, the predicted and actual values seems
    to curve upwards rather than a clean straight line that cuts
    in the middle. I need to explore basis function and log the
    values before doing prediction.
    
    # For RandomForestRegressor, the values are aligned but there are
    some outliers on test set that increase RMSE. If compared to Decision
    Tree Regressor, it is a better in generalizing and not over fit to training.
    
    # Action. Need to make this type of evaluation smoother. It is good
    to see different models in action. They are doing their own thing.
    And it is not about choosing the model, it is about how you use them.
    Linear Regression is best with numeric values only.
    While RandomForest and DecisionTree are better with IFs and Then logic.
    

In [ ]:
X = rid_reg.predict(train_pre)
Y = train[target]
Xhat = rid_reg.predict(test_pre)
Yhat = test[target]
sns.scatterplot(x=X, y=Y);
sns.scatterplot(x=Xhat, y=Yhat);

In [ ]:
X = lin_reg.predict(train_pre)
Y = train[target]
Xhat = lin_reg.predict(test_pre)
Yhat = test[target]
sns.scatterplot(x=X, y=Y);
sns.scatterplot(x=Xhat, y=Yhat);

In [ ]:
X = ran_reg.predict(train_pre)
Y = train[target]
Xhat = ran_reg.predict(test_pre)
Yhat = test[target]
sns.scatterplot(x=X, y=Y);
sns.scatterplot(x=Xhat, y=Yhat);

In [ ]:
X = dec_reg.predict(train_pre)
Y = train[target]
Xhat = dec_reg.predict(test_pre)
Yhat = test[target]
sns.scatterplot(x=X, y=Y);
sns.scatterplot(x=Xhat, y=Yhat);